# Data Extraction from API

In [1]:
import pandas as pd

## BKK Policy Tracking Webscrapping

In [2]:
bkk_url = 'https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findex'

# Read HTML tables into a list of DataFrame objects.
all_tables = pd.read_html(bkk_url)
# Check number of table(s)
len(all_tables)

1

In [3]:
df_bkk = all_tables[0]
df_bkk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   เป้าหมายตามนโยบาย ผว.กทม.  30 non-null     object 
 1   หน่วยนับ                   30 non-null     object 
 2   เป้าหมายรวม                30 non-null     float64
 3   เป้าหมายจากสำนัก           30 non-null     float64
 4   เป้าหมายจาก 50 เขต         30 non-null     float64
 5   ผลการดำเนินงาน*            30 non-null     float64
 6   ความคืบหน้า* ของ KPI (%)   31 non-null     float64
dtypes: float64(5), object(2)
memory usage: 1.8+ KB


In [4]:
pd.set_option('display.float_format', str)
pd.set_option('display.max_colwidth', None)

In [5]:
df_bkk

,เป้าหมายตามนโยบาย ผว.กทม.,หน่วยนับ,เป้าหมายรวม,เป้าหมายจากสำนัก,เป้าหมายจาก 50 เขต,ผลการดำเนินงาน*,ความคืบหน้า* ของ KPI (%)
0,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10,กม.,133.35,0.0,133.35,121.77,91.32
1,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01,ต้น,200000.0,50000.0,150000.0,290856.0,145.43
2,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05,แห่ง,156.0,6.0,150.0,118.5,75.96
3,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05,กม.,324.11,300.0,24.11,108.55,33.49
4,4.2 ทางจักรยาน (กม.) : OKR 1.3.07,กม.,518.0,18.0,500.0,200.34,38.68
5,4.3 ปรับปรุงซอย (กม.),กม.,136.0,0.0,136.0,175.21,128.84
6,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20,ดวง,32078.0,13100.0,18978.0,38967.0,121.48
7,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง) : OKR 1.1.03,ดวง,8056.0,2114.0,5942.0,4621.28,57.36
8,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง) : OKR 1.1.01,แห่ง,109.0,0.0,109.0,115.0,105.5
9,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง) : OKR 7.2.05,แห่ง,28.0,0.0,28.0,26.0,92.85


In [6]:
df_bkk.columns

Index(['เป้าหมายตามนโยบาย ผว.กทม.', 'หน่วยนับ', 'เป้าหมายรวม',
       'เป้าหมายจากสำนัก', 'เป้าหมายจาก 50 เขต', 'ผลการดำเนินงาน*',
       'ความคืบหน้า* ของ KPI (%)'],
      dtype='object')

In [7]:
df_bkk.dropna(axis = 0, inplace = True)

new_name = {'เป้าหมายตามนโยบาย ผว.กทม.': 'Key Result', 'หน่วยนับ':'Unit', 'เป้าหมายรวม': 'Total Goals',
       'เป้าหมายจากสำนัก':'Goal (Departments)', 'เป้าหมายจาก 50 เขต':'Goal (50 Districts)', 'ผลการดำเนินงาน*':'Total Progress (Unit)',
       'ความคืบหน้า* ของ KPI (%)':'Progress (%)'}

df_bkk.rename(columns = new_name, inplace = True)

# new data frame with split value columns
new = df_bkk["Key Result"].str.split(":", n=1, expand=True)

# making separate first name column from new data frame
df_bkk["Key Result"] = new[0]

# making separate last name column from new data frame
df_bkk["Related OKR"] = new[1]

# Dropping old Name columns
# df_bkk.drop(columns=["Key Result"], inplace=True)

df_bkk['Related OKR'] = df_bkk['Related OKR'].str.replace('OKR ', '')
df_bkk['Related OKR'] = df_bkk['Related OKR'].str.replace('(** ค่าเฉลี่ย **)', '(Mean Value)')

new_2 = df_bkk["Key Result"].str.split(" ", n=1, expand=True)
df_bkk["No. (Key Result)"] = new_2[0]
df_bkk["Key Result"] = new_2[1]

df_bkk = df_bkk[['No. (Key Result)', 'Related OKR', 'Key Result', 'Unit', 'Total Goals', 'Goal (Departments)',
       'Goal (50 Districts)', 'Total Progress (Unit)', 'Progress (%)']]

In [8]:
df_bkk

,No. (Key Result),Related OKR,Key Result,Unit,Total Goals,Goal (Departments),Goal (50 Districts),Total Progress (Unit),Progress (%)
0,1.,4.1.10,พัฒนาถนนสวย (กม.),กม.,133.35,0.0,133.35,121.77,91.32
1,2.,4.1.01,ปลูกต้นไม้รวมทุกประเภท (ต้น),ต้น,200000.0,50000.0,150000.0,290856.0,145.43
2,3.,4.1.05,เพิ่มสวน 15 นาที (แห่ง),แห่ง,156.0,6.0,150.0,118.5,75.96
3,4.1,1.1.05,ปรับปรุงทางเท้า (กม.),กม.,324.11,300.0,24.11,108.55,33.49
4,4.2,1.3.07,ทางจักรยาน (กม.),กม.,518.0,18.0,500.0,200.34,38.68
5,4.3,None,ปรับปรุงซอย (กม.),กม.,136.0,0.0,136.0,175.21,128.84
6,5.,2.1.20,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),ดวง,32078.0,13100.0,18978.0,38967.0,121.48
7,6.,1.1.03,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),ดวง,8056.0,2114.0,5942.0,4621.28,57.36
8,7.,1.1.01,ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง),แห่ง,109.0,0.0,109.0,115.0,105.5
9,8.,7.2.05,จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง),แห่ง,28.0,0.0,28.0,26.0,92.85


## BKK: Monthly Progress of Policy

In [65]:
# Declare Blank DataFrame
df_progress = pd.DataFrame()

# Access and concat all DataFrames to same DataFrames
for i in range(1,7):
    policy_url = "https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=" + str(i) +"&per-page=50"
    progress_table = pd.read_html(policy_url)
    df_progress = pd.concat([df_progress, progress_table[0]], ignore_index=True, axis = 0)

## Rename Columns
new_col_names = {'KEY_RESULT':'Key_Result', 'ค่าเป้าหมาย/ปี':'Yearly Goal', 'ผลดำเนินงาน (รวม)':'Acheived Goal', 'หน่วยนับ':'Unit',
       'ตค.66':'Oct 23', 'พย.66':'Nov 23', 'ธค.66':'Dec 23', 'มค.67':'Jan 24', 'กพ.67':'Feb 24', 'มี.ค.67':'Mar 24', 'เม.ย.67':'Apr 24',
       'พค.67':'May 24', 'มิ.ย.67':'Jun 24', 'กค.67':'July 24', 'สค.67':'Aug 24', 'กย.67':'Sept 24'}
df_progress.rename(columns = new_col_names, inplace = True)

# Drop Null Value
df_progress.dropna(axis = 0, inplace = True)

# Reset Index
df_progress = df_progress.reset_index()
df_progress.drop('index', axis = 1, inplace = True)

# Set Index
df_progress['#'] = df_progress['#'].astype('int')
df_progress.set_index(df_progress['#'], inplace = True)
df_progress.drop('#', axis = 1, inplace = True)

#Check DF info
df_progress.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 1 to 300
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Key_Result     300 non-null    object 
 1   Yearly Goal    300 non-null    float64
 2   Acheived Goal  300 non-null    float64
 3   Unit           300 non-null    object 
 4   Oct 23         300 non-null    float64
 5   Nov 23         300 non-null    float64
 6   Dec 23         300 non-null    float64
 7   Jan 24         300 non-null    float64
 8   Feb 24         300 non-null    float64
 9   Mar 24         300 non-null    float64
 10  Apr 24         300 non-null    float64
 11  May 24         300 non-null    float64
 12  Jun 24         300 non-null    float64
 13  July 24        300 non-null    float64
 14  Aug 24         300 non-null    float64
 15  Sept 24        300 non-null    float64
dtypes: float64(14), object(2)
memory usage: 39.8+ KB


In [66]:
df_progress

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป็นเครื่องยนต์ไฟฟ้า 100.00%,1.0,1.0,หลักสูตร,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
297,"OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถนนสายหลัก 10,000 กรง 11.91%",2250.0,268.0,กรง (สะสม),0.0,0.0,0.0,0.0,0.0,150.0,118.0,0.0,0.0,0.0,0.0,0.0
298,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร (Street Furniture) สำหรับคนเดินเท้าบน ทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,5000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
new_col_names = {'KEY_RESULT':'Key_Result', 'ค่าเป้าหมาย/ปี':'Yearly Goal', 'ผลดำเนินงาน (รวม)':'Acheived Goal', 'หน่วยนับ':'Unit',
       'ตค.66':'Oct 23', 'พย.66':'Nov 23', 'ธค.66':'Dec 23', 'มค.67':'Jan 24', 'กพ.67':'Feb 24', 'มี.ค.67':'Mar 24', 'เม.ย.67':'Apr 24',
       'พค.67':'May 24', 'มิ.ย.67':'Jun 24', 'กค.67':'July 24', 'สค.67':'Aug 24', 'กย.67':'Sept 24'}

In [63]:
df_progress.rename(columns = new_col_names, inplace = True)

In [64]:
df_progress

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป็นเครื่องยนต์ไฟฟ้า 100.00%,1.0,1.0,หลักสูตร,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
297,"OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถนนสายหลัก 10,000 กรง 11.91%",2250.0,268.0,กรง (สะสม),0.0,0.0,0.0,0.0,0.0,150.0,118.0,0.0,0.0,0.0,0.0,0.0
298,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร (Street Furniture) สำหรับคนเดินเท้าบน ทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,5000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
df_progress.set_index(df_progress['#'], inplace = True)
df_progress.drop('#', axis = 1, inplace = True)
df_progress.head()

,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
#,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0


In [45]:
df_progress.columns

Index(['#', 'KEY_RESULT', 'ค่าเป้าหมาย/ปี', 'ผลดำเนินงาน (รวม)', 'หน่วยนับ',
       'ตค.66', 'พย.66', 'ธค.66', 'มค.67', 'กพ.67', 'มี.ค.67', 'เม.ย.67',
       'พค.67', 'มิ.ย.67', 'กค.67', 'สค.67', 'กย.67'],
      dtype='object')

In [51]:
df_progress

,#,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
0,1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
1,2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
2,3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
3,4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
4,5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป็นเครื่องยนต์ไฟฟ้า 100.00%,1.0,1.0,หลักสูตร,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
296,297,"OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถนนสายหลัก 10,000 กรง 11.91%",2250.0,268.0,กรง (สะสม),0.0,0.0,0.0,0.0,0.0,150.0,118.0,0.0,0.0,0.0,0.0,0.0
297,298,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร (Street Furniture) สำหรับคนเดินเท้าบน ทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,5000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298,299,OKR 4.2.14 ติดตั้งถังขยะ 4 ประเภทขนาด 130 ลิตร (Street Furniture) สำหรับคนเดินเท้าบนทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,10000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
df_progress.reset_index()

,index,#,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
0,0,1.0,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
1,1,2.0,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
2,2,3.0,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
3,3,4.0,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
4,4,5.0,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,300,296.0,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป็นเครื่องยนต์ไฟฟ้า 100.00%,1.0,1.0,หลักสูตร,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
296,301,297.0,"OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถนนสายหลัก 10,000 กรง 11.91%",2250.0,268.0,กรง (สะสม),0.0,0.0,0.0,0.0,0.0,150.0,118.0,0.0,0.0,0.0,0.0,0.0
297,302,298.0,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร (Street Furniture) สำหรับคนเดินเท้าบน ทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,5000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298,303,299.0,OKR 4.2.14 ติดตั้งถังขยะ 4 ประเภทขนาด 130 ลิตร (Street Furniture) สำหรับคนเดินเท้าบนทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,10000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
df_progress.head()

,#,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
0,1.0,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
1,2.0,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
2,3.0,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
3,4.0,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
4,5.0,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0


In [20]:
for i in range(1,7):
    policy_url = "https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=" + str(i) +"&per-page=50"
    print(policy_url)
    print(i)

https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=1&per-page=50
1
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=2&per-page=50
2
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=3&per-page=50
3
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=4&per-page=50
4
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=5&per-page=50
5
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=6&per-page=50
6


In [15]:
progress_table[0]

,#,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
0,1.0,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
1,2.0,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
2,3.0,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
3,4.0,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
4,5.0,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0
5,6.0,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง) : OKR 1.1.03 48.62%,9536.0,4636.28,ดวง,70.0,316.0,234.0,325.0,1413.0,345.0,507.0,472.0,416.0,538.28,0.0,0.0
6,7.0,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง) : OKR 1.1.01 116.00%,100.0,116.0,แห่ง,3.0,1.0,1.0,14.0,14.0,30.0,23.0,12.0,7.0,11.0,0.0,0.0
7,8.0,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง) : OKR 7.2.05 272.73%,11.0,30.0,แห่ง,1.0,3.0,1.0,4.0,2.0,7.0,5.0,4.0,2.0,1.0,0.0,0.0
8,9.0,9. แก้ไขจุดเสี่ยงน้ำท่วม (จุด) : OKR 1.4.05 103.96%,212.0,220.4,จุด,5.0,22.0,1.0,5.0,8.01,32.0,12.2,17.2,7.45,110.54,0.0,0.0
9,10.0,10.1 แก้ปัญหาจุดจราจรฝืด (แห่ง) : OKR 1.2.05 63.77%,265.0,169.0,แห่ง,7.0,3.0,6.0,11.0,12.0,38.0,39.0,28.0,18.0,7.0,0.0,0.0


## Traffy Fondue REST API Request

In [ ]:
traffy_url = 'https://publicapi.traffy.in.th/teamchadchart-stat-api/geojson/v1?output_format=csv&name=Sukatat&org=Non&purpose=educational_project&email=skt.sukatat@gmail.com&limit=&sort=DESC'

df_traffy = pd.read_csv(traffy_url)
df_traffy.head()

,ticket_id,type,organization,organization_action,comment,coords,photo,photo_after,address,subdistrict,...,last_activity,duration_minutes_inprogress,duration_minutes_finished,duration_minutes_total,timestamp_inprogress,timestamp_finished,view_count,total_point,like,dislike
0,2024-AXHLKU,ถนน,"กรุงเทพมหานคร, เขตปทุมวัน","เขตปทุมวัน, กรุงเทพมหานคร",ปัญหาเดิมครับ อันนี้ทางเลี้ยวซ้ายผ่านตลอด แต่ม...,"100.53054,13.74574",https://storage.googleapis.com/traffy_public_b...,NaN,444 อาคาร MBK ถ. พญาไท แขวงปทุมวัน เขตปทุมวัน ...,วังใหม่,...,2024-07-25 21:53:42,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,2024-P3FQ3B,ถนน,"กรุงเทพมหานคร, เขตจตุจักร","เขตจตุจักร, กรุงเทพมหานคร",คอนโดให้จอดรถบนถนน 1 เลน ขวางการจราจร ทำให้รถต...,"100.57662,13.85703",https://storage.googleapis.com/traffy_public_b...,NaN,50/699 พหลโยธิน แขวงลาดยาว เขตจตุจักร กรุงเทพม...,ลาดยาว,...,2024-07-25 21:52:28,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,2024-LVP274,ทางเท้า,"กรุงเทพมหานคร, เขตบางกอกน้อย","เขตบางกอกน้อย, กรุงเทพมหานคร",การไฟฟ้ามาขุดทางเท้าวางสายไฟแต่ไม่เก็บงานให้เร...,"100.46957,13.75925",https://storage.googleapis.com/traffy_public_b...,NaN,453/15 ซอย จรัญสนิทวงศ์ 35 แขวงบางขุนศรี เขตบา...,บางขุนศรี,...,2024-07-25 21:51:55,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,2024-97EQYC,ถนน,"กรุงเทพมหานคร, เขตจตุจักร","เขตจตุจักร, กรุงเทพมหานคร",คอนโดให้จอดรถบนถนน 1 เลน ขวางการจราจร ทำให้รถต...,"100.57694,13.85707",https://storage.googleapis.com/traffy_public_b...,NaN,VH4G+RQ8 แขวงลาดยาว เขตจตุจักร กรุงเทพมหานคร 1...,ลาดยาว,...,2024-07-25 21:50:46,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,2024-FH9QKF,ทางเท้า,"กรุงเทพมหานคร, เขตบางซื่อ","เขตบางซื่อ, กรุงเทพมหานคร",รถยนต์ขึ้นมาจอดขวางบนทางเท้า,"100.52697,13.80626",https://storage.googleapis.com/traffy_public_b...,NaN,398/2 ถ. ประชาราษฎร์ สาย 2 แขวงบางซื่อ เขตบางซ...,บางซื่อ,...,2024-07-25 21:49:12,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [ ]:
df[:1000]

,ticket_id,type,organization,organization_action,comment,coords,photo,photo_after,address,subdistrict,...,last_activity,duration_minutes_inprogress,duration_minutes_finished,duration_minutes_total,timestamp_inprogress,timestamp_finished,view_count,total_point,like,dislike
0,ZU3CZK,อื่นๆ,"ร้องทุกข์ กทม. 1555, กรุงเทพมหานคร, เขตป้อมปรา...","เขตป้อมปราบศัตรูพ่าย, กรุงเทพมหานคร, ร้องทุกข์...",ปัญหา: ที่สำนักงานเขตป้อมปราบศัตรูพ่าย วันที่ ...,"100.51308,13.75847",https://storage.googleapis.com/traffy_public_b...,NaN,60 ถ. ศุภมิตร แขวงวัดโสมนัส เขตป้อมปราบศัตรูพ่...,วัดโสมนัส,...,2024-07-25 17:44:23,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,2024-MY4DY6,ถนน,"กรุงเทพมหานคร, เขตจตุจักร","เขตจตุจักร, กรุงเทพมหานคร",ทำท่อ ของเดิมยังขุดปิดพื้นผิวไม่เสร็จขุดเพิ่มจ...,"100.55347,13.82477",https://storage.googleapis.com/traffy_public_b...,NaN,17/6 ซอย วิภาวดีรังสิต 17 แยก 10 แขวงจตุจักร เ...,จตุจักร,...,2024-07-25 17:43:29,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,2024-83XNTW,ถนน,"กรุงเทพมหานคร, เขตบางนา","เขตบางนา, กรุงเทพมหานคร",ฝาท่อน้ำกลางถนนไม่เรียบทั้งช่วงถนน ทำให้ช่วงล่...,"100.64481,13.66191",https://storage.googleapis.com/traffy_public_b...,NaN,1 ศรีนครินทร์ 26 บางนาใต้ เขตบางนา กรุงเทพมหาน...,บางนาเหนือ,...,2024-07-25 17:42:24,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,2024-KTPQUF,อื่นๆ,"กรุงเทพมหานคร, เขตบางแค","เขตบางแค, กรุงเทพมหานคร",เอาป้ายร้านค้า “แอร์“ มาติดบนถนนสาธารณะ มีการก...,"100.37594,13.71474",https://storage.googleapis.com/traffy_public_b...,NaN,86 ซอย เศรษฐกิจ 27 แขวงบางแคเหนือ เขตบางแค กรุ...,บางแคเหนือ,...,2024-07-25 17:38:57,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,2024-KUKFDP,"อื่นๆ,วินมอไซค์","เพื่อนชัชชาติ, เขตบางกอกน้อย, กรุงเทพมหานคร","กรุงเทพมหานคร, เขตบางกอกน้อย, เพื่อนชัชชาติ",วินมอเตอร์ไซค์เก็บเกินราคา ป้าย ผู้โดยสารนั่งห...,"100.47325,13.76282",https://storage.googleapis.com/traffy_public_b...,NaN,QF7F+696 แขวงศิริราช เขตบางกอกน้อย กรุงเทพมหาน...,ศิริราช,...,2024-07-25 17:38:55,NaN,NaN,NaN,NaN,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,HBJJXK,เสียง,"ร้องทุกข์ กทม. 1555, กรุงเทพมหานคร, เขตบางรัก,...","ฝ่ายสิ่งแวดล้อมฯ เขตบางรัก, เขตบางรัก, กรุงเทพ...",ศูนย์เรื่องราวร้องทุกข์ ได้รับการประสานผ่านระบ...,"100.52348,13.73067",https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,สำนักงานเขตบางรัก ถนน นเรศ แขวงสี่พระยา เขตบาง...,สี่พระยา,...,2024-07-25 14:08:29,420.0,1046.0,1046.0,2024-07-24 22:56:39,2024-07-25 09:22:39,0,0,0,0
996,2024-6K6RNY,ถนน,"กรุงเทพมหานคร, เขตบางกอกน้อย, สน.บางกอกน้อย, ก...","เขตบางกอกน้อย, กองบังคับการตำรวจนครบาล 7 (บก.น...",รถติดมากครับ สี่แยกศิริราช,"100.48329,13.75741",https://storage.googleapis.com/traffy_public_b...,NaN,รพ.ศิริราช แขวงศิริราช เขตบางกอกน้อย กรุงเทพมห...,ศิริราช,...,2024-07-24 16:05:48,NaN,NaN,NaN,NaN,NaN,0,0,0,0
997,2024-FNXKXB,ผิดกฎจราจร,"กรุงเทพมหานคร, เขตบางกอกน้อย, สน.บางกอกน้อย, ก...","เขตบางกอกน้อย, กองบังคับการตำรวจนครบาล 7 (บก.น...",มีรถสองแถวสีแดงทะเบียน 15-4618 กรุงเทพมหานคร ว...,"100.48185,13.75972",https://storage.googleapis.com/traffy_public_b...,NaN,166/1 ถนน วัดสุทธาวาส แขวงศิริราช เขตบางกอกน้อ...,ศิริราช,...,2024-07-24 16:03:25,NaN,NaN,NaN,NaN,NaN,0,0,0,0
998,2024-NP3DBL,ไฟฟ้า,"กรุงเทพมหานคร, เขตคลองเตย, ฝ่ายโยธา เขตคลองเตย...","ฝ่ายโยธา เขตคลองเตย, เขตคลองเตย, การไฟฟ้านครหล...",เรื่องเพิ่มแสงสว่างถนน ในบริเวณซอยสุขุมวิท 40 ...,"100.58056,13.72166",https://storage.googleapis.com/traffy_public_b...,NaN,26 ซ. เหมวงศ์ แขวง พระโขนง เขตคลองเตย กรุงเทพม...,พระโขนง,...,2024-07-24 16:29:07,30.0,NaN,30.0,2024-07-24 16:25:15,NaN,0,0,0,0
